In [ ]:
import wandb
from datetime import datetime

In [ ]:
mask_model_ver = '095521'
lr = 5e-4
batch_size = 32
wd = 1e-5
IMAGE_SIZE = 768
backbone='efficientdetv2_dt'
vertical_type = "axial"
train_portion = 0.5
milestones = [10,20,30,40]
model_name = "effdet"
epochs =50



In [ ]:
wandb_entity='longyi'
model_name = "effdet"
wandb.init(project="YOLOR", entity=wandb_entity, config={
    "model":model_name,
    "batch_size":batch_size,
    "lr" : lr,
    "wd" : wd,
    "backbone" : backbone,
    "image_size" : IMAGE_SIZE,
})
wandb.run.name = f'axial_detection_{model_name}_' + datetime.now().strftime("%H%M%S")
wandb.run.name

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import sys

effdet_path = "../third/effdet"
sys.path.append(effdet_path)
timm_path = "../third/timm-pytorch-image-models"
sys.path.append(timm_path)
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from matplotlib import patches
import sklearn
import os
from tqdm import tqdm

omega_path = "../third/omegaconf"
sys.path.append(omega_path)
from omegaconf import OmegaConf
import glob
import sklearn
import math
import random

from PIL import Image

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
import torchvision.transforms as T
import torchvision.transforms.functional as TF

from effdet import create_model
from effdet import DetBenchPredict
# from transformers import get_cosine_schedule_with_warmup

import warnings

warnings.filterwarnings('ignore')
from sklearn import metrics, model_selection, preprocessing

import json
# from sklearn.model_selection import GroupKFold

%load_ext autoreload
%autoreload 2

# pos_weight = torch.tensor(pos_weight)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
DATA_DIR = "/root/autodl-tmp/cervical_spine/"
IMAGES_DIR = os.path.join(DATA_DIR, f'train_axial_images_jpeg95_croped_{mask_model_ver}')
MASK_DIR = os.path.join(DATA_DIR, f'segmentation_axial_results_{mask_model_ver}')


checkpoint = os.path.join(DATA_DIR, 'checkpoint', 'axial_detection_effdet_151039-epoch-60.pth')

In [ ]:
def get_det_model(image_size, checkpoint=None):
    if checkpoint is None:

        model = create_model(backbone , bench_task='train' , num_classes= 1 , image_size=(image_size, image_size), bench_labeler=True, pretrained=True)
        model = model.to(device)
        return model
    else:
        model = create_model(backbone , bench_task='train' , num_classes= 1 , image_size=(image_size, image_size), bench_labeler=True, pretrained=True)
        state = torch.load(os.path.join(DATA_DIR, 'checkpoint', checkpoint), map_location=torch.device(device))
        model.load_state_dict(state["model"])
        model = model.to(device)
        return model

model = get_det_model(IMAGE_SIZE, checkpoint)

In [ ]:
from evaluate import *
evaluator = Evaluate(model.model, os.path.join(DATA_DIR, 'predictions', wandb.run.name), IMAGE_SIZE, IMAGES_DIR, MASK_DIR)

In [ ]:
start_batch_index = 0
epoch=0
for i in range(252):
    loss, pos_loss, neg_loss, batch_index = evaluator.evaluate(epoch + i + start_batch_index)
    wandb.log({
        'val/total_loss': loss,
        'val/pos_loss': pos_loss,
        'val/neg_loss': neg_loss,
        'val/batch_index' : batch_index
    })